# 07. 범주형 데이터 이진 분류
## 7.4 성능 개선 I
### 7.4.1 피처 엔지니어링 I : 피처 맞춤 인코딩
### 7.4.2 피처 엔지니어링 II : 피처 스케일링
### 7.4.3 하이퍼파라미터 최적화
### 7.4.4 모델 성능 검증
### 7.4.5 예측 및 결과 제출
## 7.5 성능 개선 II

### 7.4.1 피처 엔지니어링 I : 피처 맞춤 인코딩
* 피처 맞춤 인코딩
    * ord_1, ord_2는 수작업 인코딩
    * ord_3, ord_4, ord_5는 ordinal 인코딩
    * 명목형 피처와 날짜 피처는 원핫 인코딩
* 피처 스케일링
    * 순서형 피처에 적용
* 하이퍼파라미터 최적화

In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [45]:
data_path = '../../data/07_categorical/'

train = pd.read_csv(data_path+'train.csv', index_col='id')
test = pd.read_csv(data_path+'test.csv', index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv', index_col='id')

In [46]:
# 데이터 합치기
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1)
all_data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


In [47]:
# 이진 피처 인코딩
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T': 1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y': 1})

In [48]:
# 순서형 피처 인코딩
# ord_1, ord_2
ord1dict = {'Novice': 0, 'Contributor': 1, 'Expert': 2, 'Master': 3, 'Grandmaster': 4}
ord2dict = {'Freezing': 0, 'Cold': 1, 'Warm': 2, 'Hot': 3, 'Boiling Hot': 4, 'Lava Hot': 5}
all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [49]:
# ord_3, ord_4, ord_5
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3', 'ord_4', 'ord_5']
ord_encoder = OrdinalEncoder()
all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])
for feature, categories in zip(ord_345, ord_encoder.categories_):
    print(feature)
    print(categories)

ord_3
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o']
ord_4
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
ord_5
['AP' 'Ai' 'Aj' 'BA' 'BE' 'Bb' 'Bd' 'Bn' 'CL' 'CM' 'CU' 'CZ' 'Cl' 'DH'
 'DN' 'Dc' 'Dx' 'Ed' 'Eg' 'Er' 'FI' 'Fd' 'Fo' 'GD' 'GJ' 'Gb' 'Gx' 'Hj'
 'IK' 'Id' 'JX' 'Jc' 'Jf' 'Jt' 'KR' 'KZ' 'Kf' 'Kq' 'LE' 'MC' 'MO' 'MV'
 'Mf' 'Ml' 'Mx' 'NV' 'Nf' 'Nk' 'OR' 'Ob' 'Os' 'PA' 'PQ' 'PZ' 'Ps' 'QM'
 'Qb' 'Qh' 'Qo' 'RG' 'RL' 'RP' 'Rm' 'Ry' 'SB' 'Sc' 'TR' 'TZ' 'To' 'UO'
 'Uk' 'Uu' 'Vf' 'Vx' 'WE' 'Wc' 'Wv' 'XI' 'Xh' 'Xi' 'YC' 'Yb' 'Ye' 'ZR'
 'ZS' 'Zc' 'Zq' 'aF' 'aM' 'aO' 'aP' 'ac' 'av' 'bF' 'bJ' 'be' 'cA' 'cG'
 'cW' 'ck' 'cp' 'dB' 'dE' 'dN' 'dO' 'dP' 'dQ' 'dZ' 'dh' 'eG' 'eQ' 'eb'
 'eg' 'ek' 'ex' 'fO' 'fh' 'gJ' 'gM' 'hL' 'hT' 'hh' 'hp' 'iT' 'ih' 'jS'
 'jV' 'je' 'jp' 'kC' 'kE' 'kK' 'kL' 'kU' 'kW' 'ke' 'kr' 'kw' 'lF' 'lL'
 'll' 'lx' 'mb' 'mc' 'mm' 'nX' 'nh' 'oC' 'oG' 'oH' 'oK' 'od' 'on' 'pa'
 'ps' 'qA' 'qJ' 'qK' 'qP' 'qX' '

In [50]:
# 변환 전 데이터 확인
train.head()[ord_345]

,ord_3,ord_4,ord_5
id,,,
0,h,D,kr
1,a,A,bF
2,h,R,Jc
3,i,D,kW
4,a,R,qP


In [51]:
# 변환 후 데이터 확인
all_data.head()[ord_345]

,ord_3,ord_4,ord_5
id,,,
0,7.0,3.0,136.0
1,0.0,0.0,93.0
2,7.0,17.0,31.0
3,8.0,3.0,134.0
4,0.0,17.0,158.0


In [52]:
# 명목형 피처 인코딩
from sklearn.preprocessing import OneHotEncoder
nom_features = [f'nom_{i}' for i in range(10)]

In [53]:
tmp = pd.DataFrame(index=all_data.index)
onehot_encoder = OneHotEncoder()
# tmp[nom_features] = onehot_encoder.fit_transform(all_data[nom_features])

In [54]:
onehot_encoder = OneHotEncoder()
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])
all_data = all_data.drop(columns=nom_features)
encoded_nom_matrix

<500000x16276 sparse matrix of type '<class 'numpy.float64'>'
	with 5000000 stored elements in Compressed Sparse Row format>

In [55]:
# 날짜 피처 인코딩
date_features = ['day', 'month']
encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])
all_date = all_data.drop(columns=date_features)
encoded_date_matrix

<500000x19 sparse matrix of type '<class 'numpy.float64'>'
	with 1000000 stored elements in Compressed Sparse Row format>

### 7.4.2 피처 엔지니어링 II : 피처 스케일링

In [56]:
ord_features = [f'ord_{i}' for i in range(6)]
all_data.head()[ord_features]

,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
id,,,,,,
0,2,4,1,7.0,3.0,136.0
1,1,4,3,0.0,0.0,93.0
2,1,2,5,7.0,17.0,31.0
3,1,4,4,8.0,3.0,134.0
4,1,4,0,0.0,17.0,158.0


In [57]:
# 순서형 피처 스케일링
from sklearn.preprocessing import MinMaxScaler

all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])

In [58]:
all_data.head()[ord_features]

,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
id,,,,,,
0,0.5,1.0,0.2,0.500000,0.12,0.712042
1,0.0,1.0,0.6,0.000000,0.00,0.486911
2,0.0,0.5,1.0,0.500000,0.68,0.162304
3,0.0,1.0,0.8,0.571429,0.12,0.701571
4,0.0,1.0,0.0,0.000000,0.68,0.827225


In [59]:
# 인코딩 및 스케일링된 피처 합치기
from scipy import sparse

all_data_sprs = sparse.hstack(
    [
        sparse.csr_matrix(all_data),
        encoded_nom_matrix,
        encoded_date_matrix
    ],
    format='csr'
)

In [60]:
all_data_sprs

<500000x16308 sparse matrix of type '<class 'numpy.float64'>'
	with 10163718 stored elements in Compressed Sparse Row format>

In [61]:
# 데이터 나누기
from sklearn.model_selection import train_test_split

num_train = len(train)

X_train_o = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target']

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_o, y, test_size=0.1, stratify=y, random_state=10
)

### 7.4.3 하이퍼파라미터 최적화

In [62]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
lr_params = {
    'C': [0.1, 0.125, 0.2],
    'max_iter': [800, 900, 1000],
    'solver': ['liblinear'],
    'random_state': [42]
}
gridsearch_logistic_model = GridSearchCV(estimator=logistic_model, param_grid=lr_params, scoring='roc_auc', cv=5)
gridsearch_logistic_model.fit(X_train, y_train)
print('최적 하이퍼파라미터', gridsearch_logistic_model.best_params_)

최적 하이퍼파라미터 {'C': 0.125, 'max_iter': 800, 'random_state': 42, 'solver': 'liblinear'}
CPU times: total: 4min 49s
Wall time: 5min 6s


### 7.4.4 모델 성능 검증

In [63]:
y_valid_preds = gridsearch_logistic_model.predict_proba(X_valid)[:, 1]

In [64]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_valid, y_valid_preds)
print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.8045


### 7.4.5 예측 및 결과 제출

In [65]:
y_preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_test)[:, 1]

submission['target'] = y_preds
submission.to_csv(data_path+'submission.csv')

## 7.5 성능 개선 II
* 지금까지 다룬 모델링 절차를 유지한 채로 훈련 데이터 전체를 사용해 모델을 훈련

In [66]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
lr_params = {
    'C': [0.1, 0.125, 0.2],
    'max_iter': [800, 900, 1000],
    'solver': ['liblinear'],
    'random_state': [42]
}
gridsearch_logistic_model = GridSearchCV(estimator=logistic_model, param_grid=lr_params, scoring='roc_auc', cv=5)
gridsearch_logistic_model.fit(X_train_o, y)
print('최적 하이퍼파라미터', gridsearch_logistic_model.best_params_)

y_preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_test)[:, 1]
submission['target'] = y_preds
submission.to_csv(data_path+'submission.csv')

최적 하이퍼파라미터 {'C': 0.125, 'max_iter': 800, 'random_state': 42, 'solver': 'liblinear'}
CPU times: total: 5min 35s
Wall time: 5min 50s


### 핵심 요약
1. 피처 요약표
    * 피처별 데이터 타입, 결측값 개수, 고유값 개수, 실제 입력값 등을 정리한 표
2. 타깃값 분포 확인
3. 적절한 인코딩
    * 이진 피처 : 0과 1로 인코딩
    * 명목형 피처 : 원핫 인코딩
    * 순서형 피처 : 순서에 맞게 인코딩(피처 스케일링 수행)
    * 날짜 피처 : 삼각 함수 or 원핫 인코딩
4. 피처 스케일링
    * 피처 간 값의 범위를 일치시키는 작업